In [2]:
import sys
sys.path.append('/home/longshen/work/AccGen/AccGen')

from evaluation.metrics import Metric

In [4]:
metric = Metric()
import torch
all_train_data_fp = '/data1/longshen/Datasets/Piano/POP909/latents/song_level_phr_seq_with_annot_and_instconf/train_data.pt'
train_data = torch.load(all_train_data_fp)

subset_size = 20
train_data_list = list(train_data.items())[:subset_size]
train_data_dict = dict(train_data_list)

In [ ]:
latents = [data['latent'] for data in train_data_dict.values()]
n_phrases = [data['length'] for data in train_data_dict.values()]

# pad to 512
padded_latents = []
for latent, n_phrase in zip(latents, n_phrases):
    pad_size = 512 - n_phrase
    padded_latent = torch.nn.functional.pad(latent, (0,0,0,pad_size), mode='constant', value=0)
    padded_latents.append(padded_latent)
padded_latents_tensor = torch.stack(padded_latents, dim=0)  # (B, 512, D)

In [10]:
metric.compute_fid(padded_latents_tensor, n_phrases)

4.242551729083061

In [13]:
from models.vae_inference import PhraseVAE
vae = PhraseVAE().cuda()
strs = vae.decode_batch(padded_latents_tensor.cuda())

In [15]:
from remi_z import MultiTrack
mts = [MultiTrack.from_remiz_str(s) for s in strs]

In [19]:
selected_song_ids = list(train_data_dict.keys())
for selected_id in selected_song_ids:
    metric.mel_dict.pop(selected_id, None)

In [20]:
mem_metrics = metric.calculate_memorization_metrics_batch(mts)
mem_metrics

Computing memorization metrics: 100%|██████████| 20/20 [00:15<00:00,  1.29it/s]


{'mem_rates': [0.3930635838150289,
  0.4410134600158353,
  0.37533512064343166,
  0.31143232588699077,
  0.3305993690851735,
  0.40552416823603266,
  0.41523178807947014,
  0.4312865497076024,
  0.40077569489334197,
  0.4065484311050478,
  0.3468819599109132,
  0.4061032863849765,
  0.4553759662684469,
  0.3424845573095402,
  0.42385057471264365,
  0.4729064039408867,
  0.9974293059125964,
  0.4150298889837746,
  0.4310226492793411,
  0.40827922077922074],
 'top2_ratios': [0.9969942196531792,
  0.969251890431258,
  0.995219068580024,
  0.9973583320915237,
  0.9941147001940108,
  0.9779876224638323,
  0.99629562097033,
  0.9900649518762182,
  0.9944890330714224,
  0.9938608458390177,
  0.9934423282431312,
  0.9917007564148689,
  0.9571443211859586,
  0.998892147279244,
  0.9848135525260626,
  0.9535527340286933,
  0.003984342560150042,
  0.9943704578916456,
  0.9766902637513483,
  0.9725623445703234],
 'new_sample_flags': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [21]:
srs = metric.compute_srs(padded_latents_tensor, n_phrases)
srs

0.44453125